In [1]:
import yt
import numpy as np
import os
import pandas as pd

In [2]:
curr_dir = os.getcwd()+"/"

In [3]:
def extract_parameters(input_filename):
    input_file = curr_dir+"/mu_-0.2/"+input_filename
    fo = open(input_file, "r")
    parameters = dict()
    for line in fo.readlines():
        #print(line)
        if line.startswith("nsteps"):
            temp = line.split()
            parameters["nL"] = int(temp[-1])
        elif line.startswith("n_cell"):
            temp = line.split()
            parameters["Nx"] = int(temp[2])
            parameters["Nt"] = int(temp[-1])
        elif line.startswith("nrrb.l"):
            temp = line.split()
            parameters["lambda"] = float(temp[-1])
        elif line.startswith("nrrb.mu"):
            temp = line.split()
            parameters["mu"] = float(temp[-1])
        elif line.startswith("nrrb.m"):
            temp = line.split()
            parameters["m"] = float(temp[-1])
        elif line.startswith("nrrb.w"):
            temp = line.split()
            if temp[0] == "nrrb.w_t":
                parameters["wtr"] = float(temp[-1])
            else:
                parameters["wz"] = float(temp[-1])
        elif line.startswith("nrrb.dtau"):
            temp = line.split()
            parameters["dt"] = float(temp[-1])
        elif line.startswith("nrrb.eps"):
            temp = line.split()
            parameters["eps"] = float(temp[-1])
    parameters["beta"] = parameters["dt"]*parameters["Nt"]
    fo.close()
    return parameters

In [4]:
def find_missing_files(all_steps, all_plotfiles):
    missing = []
    for step in all_steps:
        if step not in all_plotfiles:
            missing.append(step)
    return missing

In [5]:
def make_file_lists(subdir):
    all_files = os.listdir(subdir)
    all_plotfiles = []
    for filename in all_files:
        if filename.startswith("inputs"):
            input_filename = filename
    for pltfile in all_files:
        if pltfile.startswith("plt"):
            all_plotfiles.append(int(pltfile[3:]))
    parameters = extract_parameters(input_filename)
    nL = parameters['nL']
    all_steps = np.arange(0,nL,1)
    return input_filename, all_plotfiles, all_steps, parameters

In [6]:
class GridData(object):
    def __init__(self, ds):
        # store data arrays for the dataset
        cg = ds.covering_grid(left_edge=ds.domain_left_edge, dims=ds.domain_dimensions, level=0)
        coords = cg.fcoords

        xs = coords[:,0]
        ys = coords[:,1]
        ts = coords[:,2]

        self.dx = np.max(xs[1:] - xs[:-1])
        self.dy = np.max(ys[1:] - ys[:-1])
        self.dt = np.max(ts[1:] - ts[:-1])

        self.phi_1_Re = cg["phi_1_Re"][:,:,:].d
        self.phi_1_Im = cg["phi_1_Im"][:,:,:].d
        self.phi_2_Re = cg["phi_2_Re"][:,:,:].d
        self.phi_2_Im = cg["phi_2_Im"][:,:,:].d

    def get_fields(self,i,j,t):
        return complex(self.phi_1_Re[i,j,t], self.phi_1_Im[i,j,t]), complex(self.phi_2_Re[i,j,t], self.phi_2_Im[i,j,t])

Average density: $\langle \hat{n} \rangle = \frac{1}{N_{x}^{d}}\sum_{x}n_{x}$

where $n_{x}$ is the local density: $n_{x}= \frac{1}{2 N_{\tau}}e^{\beta \mu/N_{\tau}}(\phi_{1,x}-i \phi_{2,x})(\phi_{1,x-\hat{\tau}}+i \phi_{2,x-\hat{\tau}})$

Using our four complexified fields:
\begin{eqnarray}
n_{x} &=& \frac{1}{2 N_{\tau}}e^{\beta \mu/N_{\tau}}(\phi_{1,x}^{R} + i \phi_{1,x}^{I}-i \phi_{2,x}^{R} +\phi_{2,x}^{I})(\phi_{1,x-\hat{\tau}}^{R} + i \phi_{1,x-\hat{\tau}}^{I} +i \phi_{2,x-\hat{\tau}}^{R} - \phi_{2,x-\hat{\tau}}^{I})\\
&=& \frac{1}{2 N_{\tau}}e^{\beta \mu/N_{\tau}}(\phi_{1,x}^{R} + i \phi_{1,x}^{I}-i \phi_{2,x}^{R} +\phi_{2,x}^{I})(\phi_{1,x-\hat{\tau}}^{R} + i \phi_{1,x-\hat{\tau}}^{I} +i \phi_{2,x-\hat{\tau}}^{R} - \phi_{2,x-\hat{\tau}}^{I})
\end{eqnarray}

Question for self: do you need to average over all t?

Figure out how to deal with boundary conditions

In [7]:
def local_density(gd, i, j, Nt, beta, mu):
    n = complex(0.,0.)
    for t in np.arange(1,Nt):
        phi1,phi2 = gd.get_fields(i,j,t)
        phi1t,phi2t = gd.get_fields(i,j,t-1)
        n += complex(phi1, -phi2)*complex(phi1t, -phi2t)
    phi1,phi2 = gd.get_fields(i,j,0)
    phi1t,phi2t = gd.get_fields(i,j,Nt-1)
    n += complex(phi1, -phi2)*complex(phi1t, -phi2t)
    return 0.5*np.exp(-beta*mu/float(Nt))*n/float(Nt)

def total_density(gd, Nx, Nt, beta, mu):
    ntot = complex(0.,0.)
    volume = Nx**2
    for i in np.arange(0,Nx):
        for j in np.arange(0,Nx):
            ntot += local_density(gd, i, j, Nt, beta, mu)
    return ntot/volume

In [13]:
subdirectories = os.listdir(curr_dir)
df = pd.DataFrame(columns=['step','density'])
step_arr = []
dens_arr = []
for subdir in subdirectories:
    work_dir = curr_dir+subdir
    if os.path.isdir(work_dir):
        input_filename, plotfiles, steps, params = make_file_lists(subdir)
        nL_str = str(params["nL"])
        nL = 10 #params['nL']
        Nx = params['Nx']
        Nt = params['Nt']
        beta = params['beta']
        mu = params['mu']
        wz = params['wz']
        wtr = params['wtr']
        l = params['lambda']
        for step in np.arange(0,nL+1): #for step in steps:
            filename = "plt"+str(step).zfill(len(nL_str)-1)
            ds = yt.load(work_dir+"/"+filename)
            gd = GridData(ds)
            N = total_density(gd, Nx, Nt, beta, mu)
            step_arr.append(step)
            dens_arr.append(N)
            df = df.append({'step': step, 'density': N}, ignore_index=True)
    else:
        pass
df[['step']] = df[['step']].astype(int)

yt : [INFO     ] 2020-01-14 19:09:20,173 Parameters: current_time              = 0.0
yt : [INFO     ] 2020-01-14 19:09:20,174 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 19:09:20,175 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 19:09:20,176 Parameters: domain_right_edge         = [21. 21. 80.]
yt : [INFO     ] 2020-01-14 19:09:20,603 Parameters: current_time              = 0.001
yt : [INFO     ] 2020-01-14 19:09:20,604 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 19:09:20,605 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 19:09:20,606 Parameters: domain_right_edge         = [21. 21. 80.]
yt : [INFO     ] 2020-01-14 19:09:21,043 Parameters: current_time              = 0.002
yt : [INFO     ] 2020-01-14 19:09:21,044 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 19:09:21,046 Parameters: domain_left_edge          = [0. 0. 0.]

UnboundLocalError: local variable 'input_filename' referenced before assignment

In [ ]:
for s, N in zip(step_arr, dens_arr):
    print("{}: {}".format(s,N))

In [14]:
df.head()

,step,density
0,0j,(-0.0023894953363820864+0.0020966264018788265j)
1,(1+0j),(-0.0026385907472054227+0.002099770830271272j)
2,(2+0j),(-0.002399761033836866+0.001983534949230471j)
3,(3+0j),(-0.0022511918084475216+0.0015490847786933822j)
4,(4+0j),(-0.0021298563493369143+0.0012472469123865758j)


In [ ]:
def calculate_observables(parameters, work_dir):
    df = pd.DataFrame(columns=['step','N'])
    for step in np.arange(1,nL+1):
        filename = "plt"+str(step).zfill(5) 
        ds = yt.load(work_dir+filename)
        gd = GridData(ds)
        N = total_density(gd, Nx, Nt, beta, mu)
        df.append({'step': step, 'density': N}, ignore_index=True)
        step_arr.append(step)
        dens_arr.append(N)